# Laboratorio#2: Modelo Vectorial de Recuperación de Información

El Modelo  Vectorial de Recuperación de Información (MVRI) es un modelo algebraico para representar documentos de texto (o más generalmente, elementos) como vectores, de modo que la distancia entre vectores representa la relevancia entre los documentos. Es fundamental para una serie de operaciones de recuperación de información (RI), incluida la puntuación de documentos en una consulta, la clasificación de documentos y la agrupación de documentos.

Durante la clase se estará trabajando con el Modelo Vectorial. Este de define como:
- **D**:  Vectores de pesos no binarios asociados a los términos de los documentos.
- **Q**: Vectores de pesos no binarios asociados a los términos de la consulta.
- **F**: Espacio n-dimensional y operaciones entre vectores del Álgebra Lineal.
- **R**: $sim(d_j, q) = 
		 		\displaystyle{\frac{\sum_{i=1}^{n} w_{i,j} \times w_{i,q}}{\sqrt{\sum_{i=1}^{n} w_{i,j}^{2}}\times \sqrt{\sum_{i=1}^{n} w_{i,q}^{2}}}}
		 	$

In [1]:
# Configurando el entorno

# Fuente del corpus
import ir_datasets

# Facilita el trabajo con los términos indexados del corpus
from gensim.corpora import Dictionary

# Facilita la obtención de los valores tf-idf de los documentos 
from gensim.models import TfidfModel

# Para generar la matriz densa de tf-idf
from gensim.matutils import corpus2dense

# Para realizar el proceso de reduccion de dimensiones
from sklearn.decomposition import PCA

# Para generar los clusters de documentos
from sklearn.cluster import KMeans

# Funciones útiles auxiliares
from teacher_help import tokenize, cosine_similarity

### Cranfield

El corpus Cranfield es un conjunto clásico de datos en el campo de la Recuperación de Información compuesto por aproximadamente 1,400 resúmenes de artículos de investigación en aerodinámica. Cada documento en este corpus incluye un título, un resumen conciso del contenido, y en algunos casos, palabras clave y referencias bibliográficas. Acompañando a estos documentos, hay alrededor de 225 consultas de prueba y sus correspondientes juicios de relevancia, proporcionados por expertos, que indican la pertinencia de cada documento para una consulta específica. Este diseño estructurado y su enfoque específico en temas de aerodinámica hacen del corpus una herramienta esencial para evaluar la eficacia de Sistemas de Recuperación de Información (SRI), sirviendo como un modelo estándar para pruebas comparativas y consistentes en esta disciplina.

La variable **dataset**, instancia de la clase **ir_datasets.datasets.base.Dataset**, tiene 3 funciones:
1. _docs_iter()_
2. _queries_iter()_
3. _qrels_iter()_

Pero durante esta clase, solo se trabajará con la primera de ellas. La función **docs_iter()** devuelve un objeto iterable de tuplas de dimensión 5, referentes a los documentos. Los campos de cada tupla son:
  - Identificador (str)
  - Título (str)
  - Texto (str)
  - Autor (str)
  - Referencia bibliográfica (str)


---

En el laboratorio anterior se vio, entre otras cosas, el proceso de la tokenización de documentos, la generación del vocabulario y la representación de los documentos segun BoW (Bag Of Words, o bolsa de palabras). 

Nuestro primer objetivo es ejercitar los contenidos antes mencionados y preparar los datos para su posterior uso. 

### Ejercicio 1: Prepare los datos para poder trabajar.
Para ello, 

a) Cree el vocabulario del conjunto de datos.

b) Convierta a la representación **bolsa de palabras** cada documento.

###### Ayuda: Considere usar la clase `Dictionary`.

In [2]:
corpus = ir_datasets.load('cranfield')
tokenized_docs = []
for doc in corpus.docs_iter():
    tokenized_docs.append(tokenize(doc[2]))
    
  
dictionary = Dictionary(tokenized_docs)
vocabulario = list(dictionary.token2id.keys())
bolsa = [dictionary.doc2bow(doc) for doc in tokenized_docs]
print(bolsa)

[[(0, 7), (1, 1), (2, 1), (3, 1), (4, 3), (5, 1), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 5), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 2), (19, 1), (20, 3), (21, 1), (22, 2), (23, 2), (24, 1), (25, 2), (26, 1), (27, 1), (28, 2), (29, 1), (30, 1), (31, 2), (32, 1), (33, 4), (34, 1), (35, 2), (36, 1), (37, 1), (38, 1), (39, 1), (40, 4), (41, 1), (42, 2), (43, 10), (44, 1), (45, 1), (46, 2), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 5), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 12), (66, 1), (67, 1), (68, 2), (69, 5), (70, 1), (71, 1), (72, 1), (73, 1), (74, 3), (75, 2)], [(0, 9), (4, 2), (5, 2), (7, 1), (11, 10), (12, 5), (13, 2), (20, 1), (23, 1), (30, 6), (31, 1), (32, 3), (33, 7), (39, 5), (43, 6), (44, 1), (48, 4), (54, 1), (59, 3), (60, 2), (64, 2), (65, 18), (68, 1), (69, 2), (76, 1), (77, 1), (78, 1), (79, 1), (80, 2), (81, 2), (82, 1), (83, 1), (84, 2), (85, 1), (86, 1), (87, 2)

---

En el caso de la representación de los documentos en el Modelo Vectorial ae considera un modelo de espacio vectorial particular basado en la representación de una bolsa de palabras. Los documentos y consultas se representan como vectores.

$d_j = (w_{1,j},w_{2,j},\ldots,w_{n,j})$

$q = (q_{1,q},w_{2,q},\ldots,w_{n,q})$

Cada dimensión corresponde a un término o token independiente. Si un término aparece en el documento, su valor en el vector es distinto de cero. Se han desarrollado varias formas diferentes de calcular estos valores, conocidos también como ponderaciones. Uno de los esquemas más conocidos es la ponderación tf-idf.

El tf–idf es el producto de dos estadísticas: la frecuencia de los términos y la frecuencia inversa de los documentos


### Ejercicio 2: Obtenga la representación tf-idf de cada elemento del corpus y de la consulta definida.
###### Ayuda: Considere usar la clase `TfidfModel` y la función `corpus2dense`.

In [3]:
query_text = 'what similarity laws must be obeyed when constructing aeroelastic models of heated high speed aircraft .'

query_tokens = tokenize(query_text)
query_bow = dictionary.doc2bow(query_tokens)

tfidf_model = TfidfModel(bolsa)

corpus_tfidf = tfidf_model[bolsa]
query_tfidf = tfidf_model[query_bow]

corpus_tfidf_dense = corpus2dense(corpus_tfidf, len(vocabulario), len(tokenized_docs)).T
query_vector = corpus2dense([query_tfidf], len(vocabulario), 1).T

---

### Ejercicio 3: Obtenga el ranking (decreciente) de los documentos que _satisfacen_ a la consulta previamente definida.

###### Ayuda: Considere usar la función `cosine_similarity`. 

In [4]:
def retrieve_documents(corpus_matriz, vector_query):
    """
    Gets the similarity between the corpus and a query
    
    Args:
    - corpus_matriz : [[float]]
        tf-idf representation of the query. Each row is considered a document.
    - vector_query : [float]
        tf-idf representation of the query.
        
    Return:
    - [(int, float)]

    """
    
    ans = []
    
    for pos,list in enumerate(corpus_matriz):
        ans.append((pos, cosine_similarity(list, vector_query)))

    return sorted(ans, key=lambda x: x[1], reverse=True)   

print(retrieve_documents(corpus_tfidf_dense, query_vector[0])[:10])

[(183, 0.2486959), (358, 0.21426098), (50, 0.2100332), (11, 0.18004422), (55, 0.17852324), (1185, 0.16119407), (572, 0.14348215), (485, 0.14031771), (943, 0.13866155), (434, 0.1279613)]


/media/danielhp/DATA/Escuela/C. Computación/Tercer Año/SRI/Semana 3/240515-Lab_2/teacher_help.py:70: RuntimeWarning: invalid value encountered in scalar divide
  return dot(a, b)/(norm(a) * norm(b))


---

Uno de los problemas más comunes cuando se trabaja con datos muchas dimensiones es el conocido "maldición de la dimensionalidad". Este fenómeno entre otras cosas afecta nociones como distancia o similitud, haciendo que todos los objetos parezcan distantes y diferentes.

Debido a esto, una de las ideas básicas para evitar estos problemas consiste en aplicar técnicas de reducción de dimensiones. Este proceso consiste en la transformación de datos de un espacio de alta dimensión a un espacio de baja dimensión, pero en este nuevo espacio se intenta conservar algunas propiedades significativas de los datos originales, idealmente cercanas a su dimensión intrínseca.

Una de las principales técnicas lineales para la reducción de la dimensionalidad es el Análisis de Componentes Principales (PCA por sus siglas en inglés). Esta técnica realiza un mapeo lineal de los datos a un espacio de dimensiones inferiores de tal manera que se maximiza la varianza de los datos en la representación de dimensiones bajas.

### Ejercicio 4: Reduzca las dimensiones del corpus.
###### Ayuda: Considere usar la clase `PCA` y de ahí, las funciones `fit` y `transform`.

In [5]:
# Índice de varianza

variance = 0.90

def red_dim(variance):

    pca = PCA(variance)
    pca.fit(corpus_tfidf_dense)
    M = pca.transform(corpus_tfidf_dense)
    Q = pca.transform(query_vector)
    return retrieve_documents(M,Q[0])

print(red_dim(variance)[:10])

# TODO

[(183, 0.4158001), (358, 0.34989148), (50, 0.3228941), (11, 0.2997409), (55, 0.25379273), (1185, 0.23630318), (485, 0.22810033), (874, 0.18195754), (470, 0.1800087), (994, 0.1800087)]


---

Un sistema de recuperación a menudo devuelve miles de documentos en respuesta a una consulta amplia, lo que dificulta a los usuarios navegar o identificar información relevante. Una de las técnicas utilizadas para segregar parte de corpus en el análisis de una consulta son los conocidos métodos de agrupación, los cuales consisten de forma automática en asociar los documentos en una lista de categorías significativas.

Los algoritmos de agrupamiento en el análisis computacional de texto unifican esos datos en subconjuntos o grupos, asegurando que los datos de cada grupos son internamente coherentes o similares y  externamente diferentes. 

Cada grupo representa un conjunto de elementos similares entre ellos, esto se pueden utilizar para disminuir la cantidad de documentos a la hora de realizar el ranking. Para ello, se calcula la similitud de cada cluster con una consulta, para posteriormente solo utilizar aquellos documentos que pertenezcan a los grupos más similares a la consulta.

La idea del ejercicio es utilizar el metodo KMeans para generar grupos de documentos similares, y posteriormente quedarse solo con los aquellos grupos mas cercanos a la query.

# Ejercicio 5: Reduzca el espacio de búsqueda de los documentos a recuperar, trabajando con la nueva representación del corpus.

###### Ayuda: Considere usar los centroides generados por la clase `KMeans`. De igual forma, utilice las funciones  de cada cluster para encontrar la similitud

In [7]:
# Parámetros a utilizar

# Número de cluster
n_clusters = 5

# Número para filtrar la similitud mínima entre la consulta y cada centroide 
alpha = 0.4 # Puede variar

kmeans = KMeans(n_clusters)
kmeans.fit(corpus_tfidf_dense)

print(kmeans)
print(kmeans.cluster_centers_)




/home/danielhp/.local/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


KMeans(n_clusters=5)
[[ 7.7401539e-03  2.1291412e-02  2.1521044e-03 ...  1.7462298e-10
  -1.7462298e-10  1.4551915e-11]
 [ 8.8528534e-03  3.3076857e-03  3.2107621e-03 ...  2.4738256e-10
  -1.7462298e-10 -1.0186341e-10]
 [ 6.0272622e-03  4.2307656e-04  2.4090135e-03 ...  1.2699119e-03
   1.7812550e-03  1.0720023e-03]
 [ 8.0577210e-03  7.9834778e-03  4.6016737e-03 ...  0.0000000e+00
  -4.3655746e-11  4.3655746e-11]
 [ 6.7028692e-03  1.1218686e-02  2.3552559e-03 ...  4.0745363e-10
  -2.1827873e-10  1.4551915e-11]]


In [9]:
retrieve_documents(kmeans.cluster_centers_, query_vector[0])

[(4, 0.09390378),
 (1, 0.06471591),
 (0, 0.06251384),
 (3, 0.02396983),
 (2, 0.017556934)]

---
### Ejercicio 6: Genere un nuevo ranking de documentos usando solamente aquellos que pertenezcan a los grupos seleccionados en el ejercicio anterior.
###### Ayuda: Considere usar la función `KMeans.labels_`.

In [45]:
# TODO

# predict
# [(grupo_id), grupo_que_pertenezco(doc) for doc in corpus]
# para todo centroide me quedo [query - centroide] < alpha =>  {conj grupos}
#
#